<a href="https://colab.research.google.com/github/asmakhoualdia98/ortools_pb_solvers/blob/main/sudoku_cc_ob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have proto

In [1]:
!pip install python-constraint


  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24058 sha256=d2d5a7cf9d0776d3f1e65d1b5fe2e870aefa38ad6cfe5e89ef0b32833c2d6c79
  Stored in directory: /root/.cache/pip/wheels/2e/f2/2b/cb08b5fe129e4f69b7033061f256e5c551b0aa1160c2872aee
Successfully built python-constraint


In [7]:
from ortools.sat.python import cp_model

def sudoku_solver(grid):
    n = len(grid)
    m = int(n ** 0.5)

    # Créer le modèle CP-SAT
    model = cp_model.CpModel()

    # Variables représentant les valeurs des cellules
    cell_vars = {}
    for i in range(n):
        for j in range(n):
            cell_vars[(i, j)] = [model.NewBoolVar(f'cell_{i}_{j}_{k+1}') for k in range(n)]

    # Contraintes de cardinalité et opérateurs bornés

    # Contrainte 1: Chaque cellule doit avoir exactement une valeur (cardinalité)
    for i in range(n):
        for j in range(n):
            # Sum des booléens doit être égale à 1 (opérateur borné et cardinalité)
            model.Add(sum(cell_vars[(i, j)]) == 1)

    # Contrainte 2: Chaque valeur doit apparaître exactement une fois dans chaque ligne (cardinalité)
    for i in range(n):
        for k in range(n):
            # Sum des booléens pour chaque valeur dans une ligne doit être égale à 1 (opérateur borné et cardinalité)
            model.Add(sum(cell_vars[(i, j)][k] for j in range(n)) == 1)

    # Contrainte 3: Chaque valeur doit apparaître exactement une fois dans chaque colonne (cardinalité)
    for j in range(n):
        for k in range(n):
            # Sum des booléens pour chaque valeur dans une colonne doit être égale à 1 (opérateur borné et cardinalité)
            model.Add(sum(cell_vars[(i, j)][k] for i in range(n)) == 1)

    # Contrainte 4: Chaque valeur doit apparaître exactement une fois dans chaque sous-grille (cardinalité)
    for x in range(0, n, m):
        for y in range(0, n, m):
            for k in range(n):
                # Sum des booléens pour chaque valeur dans une sous-grille doit être égale à 1 (opérateur borné et cardinalité)
                model.Add(sum(cell_vars[(i, j)][k] for i in range(x, x + m) for j in range(y, y + m)) == 1)

    # Affecter les valeurs connues
    for i in range(n):
        for j in range(n):
            if grid[i][j] != 0:
                k = grid[i][j] - 1
                # Fixer la valeur connue (borne)
                model.Add(cell_vars[(i, j)][k] == 1)

    # Créer le solveur et résoudre le modèle
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Vérifier le résultat
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        # Extraire les valeurs assignées à chaque cellule
        solution = []
        for i in range(n):
            row = []
            for j in range(n):
                for k in range(n):
                    if solver.Value(cell_vars[(i, j)][k]):
                        row.append(k + 1)
                        break
            solution.append(row)
        return solution
    else:
        return None

# Exemple de grille Sudoku (0 représente les cellules vides)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Résoudre le puzzle Sudoku
solution = sudoku_solver(sudoku_grid)

# Imprimer la solution
if solution:
    for row in solution:
        print(row)
else:
    print("No solution found.")


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
